In [1]:
# ============================================================
# 0. IMPORTS
# ============================================================
import os
import json
import time
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)

In [2]:


print("=== MODEL COMPARISON: LR vs RF vs HistGradientBoosting ===")

# ============================================================
# 1. PATH HANDLING 
# ============================================================
if os.path.isdir("/data"):
    DATA_BASE = "/data"
else:
    DATA_BASE = "../data"

if os.path.isdir("/models"):
    MODELS_DIR = "/models"
else:
    MODELS_DIR = "../models"

if os.path.isdir("/output"):
    OUTPUT_DIR = "/output"
else:
    OUTPUT_DIR = "../output"

os.makedirs(OUTPUT_DIR, exist_ok=True)

VIS_DIR = os.path.join(OUTPUT_DIR, "model_comparison")
os.makedirs(VIS_DIR, exist_ok=True)

PROCESSED_DIR = os.path.join(DATA_BASE, "processed")

print("DATA_BASE   :", DATA_BASE)
print("PROCESSED   :", PROCESSED_DIR)
print("MODELS_DIR  :", MODELS_DIR)
print("OUTPUT_DIR  :", OUTPUT_DIR)
print("VIS_DIR     :", VIS_DIR)

# ============================================================
# 2. LOAD DATA (X_train, X_test, y_train, y_test)
# ============================================================
X_train_path = os.path.join(PROCESSED_DIR, "X_train.parquet")
X_test_path  = os.path.join(PROCESSED_DIR, "X_test.parquet")
y_train_path = os.path.join(PROCESSED_DIR, "y_train.parquet")
y_test_path  = os.path.join(PROCESSED_DIR, "y_test.parquet")

print("\nLoading prepared data...")
X_train = pd.read_parquet(X_train_path)
X_test  = pd.read_parquet(X_test_path)

y_train_df = pd.read_parquet(y_train_path)
y_test_df  = pd.read_parquet(y_test_path)

if y_train_df.shape[1] == 1:
    y_train = y_train_df.iloc[:, 0]
else:
    raise ValueError("y_train has more than 1 column; please adjust.")

if y_test_df.shape[1] == 1:
    y_test = y_test_df.iloc[:, 0]
else:
    raise ValueError("y_test has more than 1 column; please adjust.")

print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

feature_names = X_train.columns.tolist()

# ============================================================
# 3. LOAD TRAINED MODELS
# ============================================================
print("\nLoading trained models from:", MODELS_DIR)

lr_path   = os.path.join(MODELS_DIR, "linear_regression_model.pkl")
rf_path   = os.path.join(MODELS_DIR, "random_forest_model.pkl")
hgbm_path = os.path.join(MODELS_DIR, "hist_gbm_model.pkl")  

models = {}

if os.path.exists(lr_path):
    models["Linear Regression"] = joblib.load(lr_path)
    print("✔ Loaded Linear Regression")
else:
    raise FileNotFoundError(f"Linear Regression model not found at {lr_path}")

if os.path.exists(rf_path):
    models["Random Forest"] = joblib.load(rf_path)
    print("✔ Loaded Random Forest")
else:
    raise FileNotFoundError(f"Random Forest model not found at {rf_path}")

if os.path.exists(hgbm_path):
    models["HistGradientBoosting"] = joblib.load(hgbm_path)
    print("✔ Loaded HistGradientBoostingRegressor")
else:
    raise FileNotFoundError(
        f"HistGradientBoosting model not found at {hgbm_path}\n"
        "Train it first (PROMPT_10 replacement)."
    )

# ============================================================
# 4. METRIC HELPERS
# ============================================================
def mape(y_true, y_pred, eps=1e-6):
    """Mean Absolute Percentage Error in % (ignore near-zero actuals)."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = np.abs(y_true) > eps
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0

def compute_metrics(y_true_train, y_pred_train,
                    y_true_test, y_pred_test):
    """Return metrics dict for train & test."""
    # Train
    rmse_train = np.sqrt(mean_squared_error(y_true_train, y_pred_train))
    mae_train  = mean_absolute_error(y_true_train, y_pred_train)
    mape_train = mape(y_true_train, y_pred_train)
    r2_train   = r2_score(y_true_train, y_pred_train)

    # Test
    rmse_test = np.sqrt(mean_squared_error(y_true_test, y_pred_test))
    mae_test  = mean_absolute_error(y_true_test, y_pred_test)
    mape_test = mape(y_true_test, y_pred_test)
    r2_test   = r2_score(y_true_test, y_pred_test)

    return {
        "Train_RMSE": rmse_train,
        "Test_RMSE": rmse_test,
        "Train_MAE": mae_train,
        "Test_MAE": mae_test,
        "Train_MAPE": mape_train,
        "Test_MAPE": mape_test,
        "Train_R2": r2_train,
        "Test_R2": r2_test,
    }

def load_training_time(model_key):
    """
    Try to load training time from JSON in MODELS_DIR.
    Expected filenames:
      - linear_regression_time.json
      - random_forest_time.json
      - hist_gbm_time.json
    """
    fname_map = {
        "Linear Regression": "linear_regression_time.json",
        "Random Forest": "random_forest_time.json",
        "HistGradientBoosting": "hist_gbm_time.json",
    }
    fname = fname_map.get(model_key)
    if fname is None:
        return np.nan
    path = os.path.join(MODELS_DIR, fname)
    if os.path.exists(path):
        try:
            with open(path, "r") as f:
                data = json.load(f)
            return float(data.get("training_time_seconds", np.nan))
        except Exception:
            return np.nan
    return np.nan

# ============================================================
# 5. EVALUATE ALL MODELS
# ============================================================
results = []
predictions = {}   
residuals  = {}    

print("\nEvaluating models...")

for name, model in models.items():
    print(f"\n--- {name} ---")

    train_time = load_training_time(name)

    t0 = time.time()
    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)
    pred_time = time.time() - t0

    predictions[name] = y_pred_test
    residuals[name]   = y_test.values - y_pred_test

    # Metrics
    metrics = compute_metrics(y_train, y_pred_train, y_test, y_pred_test)
    metrics["Model"] = name
    metrics["Train_Time_s"] = train_time
    metrics["Pred_Time_s"]  = pred_time

    results.append(metrics)

results_df = pd.DataFrame(results).set_index("Model")
comparison_csv_path = os.path.join(OUTPUT_DIR, "model_comparison.csv")
results_df.to_csv(comparison_csv_path)
print("\nModel comparison table saved to:", comparison_csv_path)
print(results_df)

# ============================================================
# 6. VISUAL COMPARISON – METRICS
# ============================================================
def bar_compare(metric_name, ylabel, filename):
    plt.figure(figsize=(8, 5))
    ax = results_df[[f"Train_{metric_name}", f"Test_{metric_name}"]].plot(
        kind="bar",
        figsize=(8, 5)
    )
    plt.title(f"{metric_name} comparison (Train vs Test)")
    plt.ylabel(ylabel)
    plt.xticks(rotation=20, ha="right")
    plt.tight_layout()
    out_path = os.path.join(VIS_DIR, filename)
    plt.savefig(out_path)
    plt.close()
    print("Saved:", out_path)

bar_compare("RMSE", "RMSE", "rmse_comparison.png")
bar_compare("MAE",  "MAE",  "mae_comparison.png")
bar_compare("R2",   "R² score", "r2_comparison.png")

# Training time comparison
plt.figure(figsize=(6, 4))
results_df["Train_Time_s"].plot(kind="bar")
plt.ylabel("Training time (s)")
plt.title("Training Time Comparison")
plt.xticks(rotation=20, ha="right")
plt.tight_layout()
tt_path = os.path.join(VIS_DIR, "training_time_comparison.png")
plt.savefig(tt_path)
plt.close()
print("Saved:", tt_path)

# ============================================================
# 7. STATISTICAL ANALYSIS BY FARE RANGE
# ============================================================
print("\nAverage absolute error by fare range:")
fare_ranges = {
    "Low (0–15]":   (0, 15),
    "Medium (15–50]": (15, 50),
    "High (50+)":   (50, np.inf),
}

fare_range_rows = []

for model_name, y_pred in predictions.items():
    for range_name, (low, high) in fare_ranges.items():
        mask = (y_test > low) & (y_test <= high)
        if mask.sum() == 0:
            mean_abs_err = np.nan
        else:
            mean_abs_err = np.mean(
                np.abs(y_test[mask].values - y_pred[mask])
            )
        fare_range_rows.append({
            "Model": model_name,
            "FareRange": range_name,
            "MeanAbsError": mean_abs_err
        })

fare_range_df = pd.DataFrame(fare_range_rows)
fare_range_path = os.path.join(OUTPUT_DIR, "fare_range_errors.csv")
fare_range_df.to_csv(fare_range_path, index=False)
print(fare_range_df)
print("Fare range error table saved to:", fare_range_path)

# ============================================================
# 8. RESIDUAL COMPARISON
# ============================================================
# Residual histograms
plt.figure(figsize=(10, 6))
for i, (name, res) in enumerate(residuals.items()):
    plt.hist(res, bins=80, alpha=0.4, label=name)
plt.xlabel("Residual (y_true - y_pred)")
plt.ylabel("Count")
plt.title("Residual Distribution Comparison")
plt.legend()
plt.tight_layout()
res_hist_path = os.path.join(VIS_DIR, "residual_histograms.png")
plt.savefig(res_hist_path)
plt.close()
print("Saved:", res_hist_path)

# Residual vs predicted plots (separate figure with subplots)
plt.figure(figsize=(12, 8))
for i, (name, y_pred) in enumerate(predictions.items(), start=1):
    plt.subplot(2, 2, i)
    plt.scatter(y_pred, residuals[name], s=2, alpha=0.3)
    plt.axhline(0, color="red", linestyle="--", linewidth=1)
    plt.xlabel("Predicted fare")
    plt.ylabel("Residual")
    plt.title(name)
plt.tight_layout()
res_vs_pred_path = os.path.join(VIS_DIR, "residuals_vs_predicted.png")
plt.savefig(res_vs_pred_path)
plt.close()
print("Saved:", res_vs_pred_path)

# ============================================================
# 9. PREDICTION AGREEMENT (CORRELATION BETWEEN MODELS)
# ============================================================
pred_df = pd.DataFrame(predictions)
corr_pred = pred_df.corr()
corr_pred_path = os.path.join(OUTPUT_DIR, "prediction_correlations.csv")
corr_pred.to_csv(corr_pred_path)
print("\nPrediction correlations between models:")
print(corr_pred)
print("Prediction correlation matrix saved to:", corr_pred_path)

# ============================================================
# 10. FEATURE IMPORTANCE COMPARISON
# ============================================================
fi_rows = []

# Linear Regression: absolute coefficients
lr_model = models["Linear Regression"]
if hasattr(lr_model, "coef_"):
    lr_coefs = np.abs(lr_model.coef_)
    for feat, val in zip(feature_names, lr_coefs):
        fi_rows.append({
            "Feature": feat,
            "Model": "Linear Regression",
            "Importance": val
        })

# Random Forest: feature_importances_
rf_model = models["Random Forest"]
if hasattr(rf_model, "feature_importances_"):
    rf_imp = rf_model.feature_importances_
    for feat, val in zip(feature_names, rf_imp):
        fi_rows.append({
            "Feature": feat,
            "Model": "Random Forest",
            "Importance": val
        })

# HistGradientBoosting: feature_importances_
hgbm_model = models["HistGradientBoosting"]
if hasattr(hgbm_model, "feature_importances_"):
    hgbm_imp = hgbm_model.feature_importances_
    for feat, val in zip(feature_names, hgbm_imp):
        fi_rows.append({
            "Feature": feat,
            "Model": "HistGradientBoosting",
            "Importance": val
        })

fi_df = pd.DataFrame(fi_rows)
fi_path = os.path.join(OUTPUT_DIR, "feature_importance_comparison.csv")
fi_df.to_csv(fi_path, index=False)
print("\nFeature importance comparison saved to:", fi_path)

avg_imp_df = (
    fi_df
    .groupby(["Model"])
    .apply(lambda g: g.assign(NormImp=g["Importance"] / g["Importance"].sum()))
    .reset_index(drop=True)
)
avg_imp = (
    avg_imp_df.groupby("Feature")["NormImp"]
    .mean()
    .sort_values(ascending=False)
)
top20_features = avg_imp.head(20)

plt.figure(figsize=(10, 6))
top20_features[::-1].plot(kind="barh")  
plt.xlabel("Average normalized importance (across models)")
plt.title("Top 20 Features (Average Importance)")
plt.tight_layout()
fi_top_path = os.path.join(VIS_DIR, "top20_features_avg_importance.png")
plt.savefig(fi_top_path)
plt.close()
print("Saved:", fi_top_path)

# ============================================================
# 11. OVERFITTING ANALYSIS & BEST MODEL SELECTION
# ============================================================
results_df["RMSE_Gap"] = results_df["Test_RMSE"] - results_df["Train_RMSE"]

best_idx = results_df["Test_RMSE"].idxmin()
best_row = results_df.loc[best_idx]

best_model_name = best_idx
best_test_rmse = best_row["Test_RMSE"]
best_train_rmse = best_row["Train_RMSE"]
best_gap = best_row["RMSE_Gap"]

print("\n=== BEST MODEL SELECTION ===")
print("Best model (lowest Test RMSE):", best_model_name)
print("Test RMSE:", best_test_rmse)
print("Train RMSE:", best_train_rmse)
print("RMSE Gap (Test - Train):", best_gap)

worst_gap_idx = results_df["RMSE_Gap"].idxmax()
print("\nModel with largest Train-Test RMSE gap (most overfitting):", worst_gap_idx)

# ============================================================
# 12. SUMMARY REPORT
# ============================================================
report_path = os.path.join(OUTPUT_DIR, "model_selection_report.txt")
with open(report_path, "w") as f:
    f.write("MODEL COMPARISON & SELECTION REPORT\n")
    f.write("===================================\n\n")
    f.write("Comparison table (metrics):\n")
    f.write(results_df.to_string())
    f.write("\n\nOverfitting analysis (RMSE Gap = Test_RMSE - Train_RMSE):\n")
    f.write(results_df["RMSE_Gap"].to_string())
    f.write("\n\nBest model:\n")
    f.write(f"- Name       : {best_model_name}\n")
    f.write(f"- Test RMSE  : {best_test_rmse:.4f}\n")
    f.write(f"- Train RMSE : {best_train_rmse:.4f}\n")
    f.write(f"- RMSE Gap   : {best_gap:.4f}\n\n")

    f.write("Fare range error summary (Mean Absolute Error):\n")
    f.write(fare_range_df.to_string(index=False))
    f.write("\n\nPrediction correlations between models:\n")
    f.write(corr_pred.to_string())
    f.write("\n\nKey insights:\n")
    f.write("- The best model is selected using lowest Test RMSE.\n")
    f.write("- RMSE gap helps diagnose overfitting (large gap = overfit).\n")
    f.write("- Fare-range errors show which model works better for low vs high fares.\n")
    f.write("- Prediction correlations indicate how similarly the models behave.\n")
    f.write("\nRecommendations:\n")
    f.write("- Use the best model above for deployment.\n")
    f.write("- Consider further hyperparameter tuning on the best model.\n")
    f.write("- Investigate high-residual trips (outliers) for data quality issues.\n")

print("\nSummary report saved to:", report_path)
print("\n Model comparison & selection completed.")


=== MODEL COMPARISON: LR vs RF vs HistGradientBoosting ===
DATA_BASE   : ../data
PROCESSED   : ../data/processed
MODELS_DIR  : ../models
OUTPUT_DIR  : ../output
VIS_DIR     : ../output/model_comparison

Loading prepared data...
X_train: (1568133, 57) X_test: (392034, 57)
y_train: (1568133,) y_test: (392034,)

Loading trained models from: ../models
✔ Loaded Linear Regression
✔ Loaded Random Forest
✔ Loaded HistGradientBoostingRegressor

Evaluating models...

--- Linear Regression ---

--- Random Forest ---

--- HistGradientBoosting ---

Model comparison table saved to: ../output/model_comparison.csv
                      Train_RMSE  Test_RMSE  Train_MAE  Test_MAE  Train_MAPE  \
Model                                                                          
Linear Regression       5.307594  12.508668   2.116691  2.196437   21.452854   
Random Forest           1.656948   2.116527   0.403223  0.559040   12.437571   
HistGradientBoosting    2.230088   2.318082   0.623631  0.636380   16.2609

/var/folders/c_/mhx6w8j52_d7vbz5h9bq_djc0000gn/T/ipykernel_22159/1742821039.py:352: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.assign(NormImp=g["Importance"] / g["Importance"].sum()))


<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>